## Data science assignment #2

데이터 사이언스 과제#2는 주어진 `train set`을 사용하여 decision tree를 만들어 `test set`에 잘 들어맞게 만드는 것이다.
먼저 주어진 데이터들을 확인해 보자

In [5]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv('./data/dt_train1.txt', sep='\t')
train.head(3)

,buying,maint,doors,persons,lug_boot,safety,car_evaluation
0,high,high,3,4,big,low,unacc
1,med,high,2,2,small,med,unacc
2,low,med,5more,2,big,high,unacc


In [7]:
test = pd.read_csv('./data/dt_test1.txt', sep='\t')
test.head(3)

,buying,maint,doors,persons,lug_boot,safety
0,med,vhigh,2,4,med,med
1,low,high,4,4,small,low
2,high,vhigh,4,4,med,med


In [8]:
answer = pd.read_csv('./test/dt_answer1.txt', sep='\t')
answer.head(3)

,buying,maint,doors,persons,lug_boot,safety,car_evaluation
0,med,vhigh,2,4,med,med,unacc
1,low,high,4,4,small,low,unacc
2,high,vhigh,4,4,med,med,unacc


위에서 데이터들이 어떠한 칼럼을 가지고 있고, 어떤 값을 가지고 있는지 확인할 수 있다.

가장 먼저 확인해야 할 일은 데이터들이 어떠한 특징을 가지고 있는지이다. 먼저 categorical data는 다루기 힘들기 때문에 numeric으로 바꿔주자

In [9]:
# 우리가 예측해야하는 칼럼의 이름을 y_label이라고 하자
y_label = list(set(train.columns) - set(test.columns))[0]

In [10]:
# 각 칼럼에 pandas.factorize를 적용함으로써 쉽게 numeric으로 바꿀 수 있다.
df = pd.concat([train, test, answer]).apply(lambda x: pd.factorize(x)[0])
df.head(3)

,buying,car_evaluation,doors,lug_boot,maint,persons,safety
0,0,0,0,0,0,0,0
1,1,0,1,1,0,1,1
2,2,0,2,0,1,1,2


In [11]:
# factorize를 적용하기 위해 train, test, answer을 합쳤으므로, 다시 나눠준다.
x_train = df.iloc[:len(train)].drop(y_label, axis=1)
y_train = df.iloc[:len(train)][y_label]
x_test = df.iloc[len(train):len(train)+len(test)].drop(y_label, axis=1)
ans = df.iloc[len(train)+len(test):][y_label]

In [18]:
assert x_test.shape == test.shape
assert len(ans) == len(answer)

여러 Classifier로 각각 어떤 결과들이 나오는지 확인해보자

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.svm import SVC

In [30]:
state = np.random.RandomState(1000)
models = [DecisionTreeClassifier(random_state=state),
          RandomForestClassifier(n_estimators=256, random_state=1000),
          ExtraTreesClassifier(random_state=state),
          AdaBoostClassifier(DecisionTreeClassifier(random_state=state)),
          BaggingClassifier(random_state=state),
          ExtraTreeClassifier(random_state=state),
          GradientBoostingClassifier(random_state=state),
         ]

In [31]:
def process(classifier):
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    count = sum([pred == ans for pred, ans in zip(y_pred, ans)])
    return count

In [32]:
for model in models:
    r = process(model)
    print (model.__class__, r)

<class 'sklearn.tree.tree.DecisionTreeClassifier'> 328
<class 'sklearn.ensemble.forest.RandomForestClassifier'> 322
<class 'sklearn.ensemble.forest.ExtraTreesClassifier'> 299
<class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'> 328
<class 'sklearn.ensemble.bagging.BaggingClassifier'> 329
<class 'sklearn.tree.tree.ExtraTreeClassifier'> 275
<class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'> 327
